In [1]:
!pip install -q h5py pyyaml
!pip install -q tf_nightly

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
tensorflow 1.13.1 has requirement tensorboard<1.14.0,>=1.13.0, but you'll have tensorboard 1.14.0 which is incompatible.
Cannot uninstall 'wrapt'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

/Users/i863659/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.13.1'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 1s 0us/step


In [4]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model


# Create a basic model instance
model = create_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 896/1000 [=========================>....] - ETA: 0s - loss: 1.2063 - acc: 0.6596
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
1000/1000 [==============================] - 1s 512us/sample - loss: 1.1447 - acc: 0.6730 - val_loss: 0.7028 - val_acc: 0.7930
Epoch 2/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.4172 - acc: 0.8958
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 195us/sample - loss: 0.4153 - acc: 0.8910 - val_loss: 0.5139 - val_acc: 0.8400
Epoch 3/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.2999 - acc: 0.9234
Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow opt

In [6]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 110us/sample - loss: 2.3653 - acc: 0.1040
Untrained model, accuracy: 10.40%


In [7]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 34us/sample - loss: 0.4170 - acc: 0.8730
Restored model, accuracy: 87.30%


In [8]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00005: saving model to training_2/cp-0005.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00010: saving model to training_2/cp-0010.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00015: saving model to training_2/cp-0015.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00020: saving model to training_2/cp-0020.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00025: saving model to training_2/cp-0025.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00030: saving model to training_2/cp-0030.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00035: saving model to training_2/cp-0035.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00040: saving model to training_2/cp-0040.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00045: saving model to training_2/cp-0045.ckpt

Consider 

In [9]:
! ls {checkpoint_dir}

checkpoint                       cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001 cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index               cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001 cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index               cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001 cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index               cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001 cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index               cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001 cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index               cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [14]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [15]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 104us/sample - loss: 0.4918 - acc: 0.8710
Restored model, accuracy: 87.10%


In [16]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 122us/sample - loss: 0.4918 - acc: 0.8710
Restored model, accuracy: 87.10%


In [17]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 359us/sample - loss: 1.1588 - acc: 0.6700
Epoch 2/5
1000/1000 [==============================] - 0s 153us/sample - loss: 0.4108 - acc: 0.8860
Epoch 3/5
1000/1000 [==============================] - 0s 152us/sample - loss: 0.2911 - acc: 0.9230
Epoch 4/5
1000/1000 [==============================] - 0s 159us/sample - loss: 0.2049 - acc: 0.9540
Epoch 5/5
1000/1000 [==============================] - 0s 153us/sample - loss: 0.1588 - acc: 0.9690


In [18]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 168us/sample - loss: 0.4352 - acc: 0.8560
Restored model, accuracy: 85.60%


In [20]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 378us/sample - loss: 1.1940 - acc: 0.6400
Epoch 2/5
1000/1000 [==============================] - 0s 145us/sample - loss: 0.4304 - acc: 0.8780
Epoch 3/5
1000/1000 [==============================] - 0s 155us/sample - loss: 0.2913 - acc: 0.9180
Epoch 4/5
1000/1000 [==============================] - 0s 159us/sample - loss: 0.2097 - acc: 0.9520
Epoch 5/5
1000/1000 [==============================] - 0s 199us/sample - loss: 0.1579 - acc: 0.9620


In [33]:
import time

saved_model_path = "./saved_models/"+str(int(time.time()))
saved_model_path_final = tf.contrib.saved_model.save_keras_model(model, saved_model_path)


Consider using a TensorFlow optimizer from `tf.train`.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_models/1562899701/1562899701/saved_model.pb


In [34]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path_final)
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [35]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 158us/sample - loss: 0.4173 - acc: 0.8590
Restored model, accuracy: 85.90%
